# Import libraries and prepare data

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

import dask
import dask.array as da
import dask.dataframe as dd
from IPython.display import Markdown
import cmdstanpy
import tempfile
from trendfilter import trend_filter

In [2]:
%%time
import polars as pl
test_series = (pl.scan_parquet('test_series.parquet')
                .with_columns(
                    (
                        (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")),
#                         (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.year().alias("year")),
#                         (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.month().alias("month")),
#                         (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.day().alias("day")),
#                         (pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z").dt.hour().alias("hour")),
                    )
                )
                .collect()
                .to_pandas()
               )

CPU times: user 25 ms, sys: 19.3 ms, total: 44.2 ms
Wall time: 94.8 ms


In [3]:
from pandas.api.types import is_datetime64_ns_dtype
import gc

import warnings
warnings.filterwarnings("ignore")

def reduce_mem_usage(df):
    
    """ 
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and not is_datetime64_ns_dtype(df[col]):
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        

    df['series_id'] = df['series_id'].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f'Decreased by {decrease:.2f}%')
    
    return df

In [4]:
test_series = reduce_mem_usage(test_series)

Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by 23.71%


In [5]:
series_ids = test_series['series_id'].unique()

# Model

In [6]:
stan_model = """
data {
  int T;
  vector[T] y;
}
parameters {
  // real<lower = 0> sigma;
  // real<lower = 0> tau;
  real<lower = 0, upper = 1> xi1_init; 
}
transformed parameters {
  matrix[T, 2] eta;
  matrix[T, 2] xi;
  vector[T] f;
  
  // fill in etas
  for(t in 1:T) {
    if(t==1) {
      eta[t,1] = exp(normal_lpdf(y[t]| 0, 20));    
      eta[t,2] = exp(normal_lpdf(y[t]| 0, 20));
    } else {
      eta[t,1] = exp(normal_lpdf(y[t]| 0.9*y[t-1], 20));
      eta[t,2] = 0.99 * exp(normal_lpdf(y[t]| y[t-1], .1)) + 0.01 * exp(normal_lpdf(y[t]| 0.9*y[t-1], 20));
    }
  }
  
  // work out likelihood contributions
  
  for(t in 1:T) {
    // for the first observation
    if(t==1) {
      f[t] = 0.999*xi1_init*eta[t,1] + // stay in state 1
             (1 - 0.999)*xi1_init*eta[t,2] + // transition from 1 to 2
             0.999*(1 - xi1_init)*eta[t,2] + // stay in state 2 
             (1 - 0.999)*(1 - xi1_init)*eta[t,1]; // transition from 2 to 1
      
      xi[t,1] = (0.999*xi1_init*eta[t,1] +(1 - 0.999)*(1 - xi1_init)*eta[t,1])/f[t];
      xi[t,2] = 1.0 - xi[t,1];
    
    } else {
    // and for the rest
      
      f[t] = 0.999*xi[t-1,1]*eta[t,1] + // stay in state 1
             (1 - 0.999)*xi[t-1,1]*eta[t,2] + // transition from 1 to 2
             0.999*xi[t-1,2]*eta[t,2] + // stay in state 2 
             (1 - 0.999)*xi[t-1,2]*eta[t,1]; // transition from 2 to 1
      
      // work out xi
      
      xi[t,1] = (0.999*xi[t-1,1]*eta[t,1] +(1 - 0.999)*xi[t-1,2]*eta[t,1])/f[t];
      
      // there are only two states so the probability of the other state is 1 - prob of the first
      xi[t,2] = 1.0 - xi[t,1];
    }
  }
  
}
model {
  // priors
  // sigma ~ cauchy(0, 10);
  // tau ~ cauchy(0, 10);
  xi1_init ~ beta(2, 2);  
  
  target += sum(log(f));
}
"""

In [7]:
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.stan') as stan_file:
    stan_file.write(stan_model)
    stan_file_path = stan_file.name

In [8]:
model = cmdstanpy.CmdStanModel(stan_file=stan_file_path)

19:04:46 - cmdstanpy - INFO - compiling stan file /tmp/tmp5k8kuxih.stan to exe file /tmp/tmp5k8kuxih
19:05:07 - cmdstanpy - INFO - compiled model executable: /tmp/tmp5k8kuxih


# Fit

In [31]:
output = pd.DataFrame(columns = ['series_id', 'step', 'event', 'score'])

for i in range(len(series_ids)):
    test_data = test_series[test_series.series_id == series_ids[i]]
    length = test_data.shape[0]
    data = {"T": length, 'y' : np.array(test_data.anglez)}
    mle = model.optimize(data=data)
    pred = np.round(mle.stan_variable('xi')[:, 0])
    pred_new = pred
    
    x = np.linspace(0, len(pred), len(pred))
    alpha = 10
    while ((length * 5)/3600/24 + 1 < sum(pred_new[1:] != pred_new[:-1])):
        tf = trend_filter(x, pred, l_norm = 1, alpha_1 = alpha + 10)
        pred_new = np.round(tf['y_fit'])
        
    
    onsets = sum((pred_new[1:] - pred_new[:-1]) == -1)
    if onsets > 0:
        output = output.append(test_data.iloc[1:, :].iloc[(pred[1:] - pred[:-1]) == -1, 0:2], ignore_index = True)
        output['event'][(len(output.index)-onsets) : ] = 'onset'

    wakeups = sum((pred_new[1:] - pred_new[:-1]) == 1)
    if wakeups > 0:
        output = output.append(test_data.iloc[1:, :].iloc[(pred[1:] - pred[:-1]) == 1, 0:2], ignore_index = True)
        output['event'][(len(output.index)-wakeups) : ] = 'wakeup'

    output['score'] = 1.0

19:18:01 - cmdstanpy - INFO - Chain [1] start processing
19:18:01 - cmdstanpy - INFO - Chain [1] done processing
19:18:01 - cmdstanpy - INFO - Chain [1] start processing
19:18:01 - cmdstanpy - INFO - Chain [1] done processing
19:18:01 - cmdstanpy - INFO - Chain [1] start processing
19:18:01 - cmdstanpy - INFO - Chain [1] done processing


In [32]:
output.to_csv('submission.csv', index_label = 'row_id')